In [54]:
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import Perceptron
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
RANDOM_STATE = 11
RS = np.random.RandomState(RANDOM_STATE)

## Load Features

In [55]:
train_ = np.load('output/train_features.npz')
val_ = np.load('output/val_features.npz')
test_ = np.load('output/test_features.npz')

train_X, train_y = train_['X'], train_['y']
val_X, val_y = val_['X'], val_['y']
test_X, test_y = test_['X'], test_['y']

In [56]:
print("{:<8} shape: ".format('train_X'), train_X.shape)
print("{:<8} shape: ".format('val_X'), val_X.shape)
print("{:<8} shape: ".format('test_X'), test_X.shape)

train_X  shape:  (63325, 1344)
val_X    shape:  (450, 1344)
test_X   shape:  (450, 1344)


## Build Model
<pre>
Choose at least three methods: Perceptron, Catboost, PyTorch-nn
Show the curves of accuracy in trainning and testing phase.
The predict result on the validation/testing phase. (top-1 accuracy and top-5 accuracy)
</pre>

### Toolbox

In [57]:
def calc_top5_acc(true_y, top5_y):
    ls_correct = [1 if true_y[i] in top5_y[i] else 0 for i in range(len(true_y))]
    return sum(ls_correct) / len(ls_correct)

def evaluate_score(true_y, pred_y_proba):
    pred_y = pred_y_proba.argmax(axis=1)
    top1_acc = round(accuracy_score(true_y, pred_y), 4)
    
    top5_y = np.argpartition(pred_y_proba, -5)[:, -5:]
    top5_acc = round(calc_top5_acc(true_y, top5_y), 4)
    return (top1_acc, top5_acc)

# evaluate_score(val_y, val_y_proba)

### Perceptron

In [58]:
per_clf = Perceptron(random_state=RANDOM_STATE, n_jobs=-1, shuffle=True)
per_clf.fit(train_X, train_y)
val_y_proba = per_clf.decision_function(val_X)

test_y_proba = per_clf.decision_function(test_X)

val_eval = evaluate_score(val_y, val_y_proba)
test_eval = evaluate_score(test_y, test_y_proba)

# top1_acc, top5_acc
print(val_eval)
print(test_eval)

(0.0422, 0.1822)
(0.0644, 0.1778)


### CatBoost

In [6]:
cat_clf = CatBoostClassifier(
    iterations=2000, early_stopping_rounds=50,
    random_state=RANDOM_STATE, verbose=True, use_best_model=True)
cat_clf.fit(train_X, train_y, eval_set=(val_X, val_y))
val_y_proba = cat_clf.predict_proba(val_X)
test_y_proba = cat_clf.predict_proba(test_X)

val_eval = evaluate_score(val_y, val_y_proba)
test_eval = evaluate_score(test_y, test_y_proba)

# top1_acc, top5_acc
print(val_eval)
print(test_eval)

# output 
# (0.2978, 0.6133)
# (0.28, 0.5822)

Learning rate set to 0.090026
0:	learn: 3.8669413	test: 3.8646928	best: 3.8646928 (0)	total: 1.13s	remaining: 37m 31s
1:	learn: 3.8290466	test: 3.8213387	best: 3.8213387 (1)	total: 2.2s	remaining: 36m 36s
2:	learn: 3.7907460	test: 3.7811094	best: 3.7811094 (2)	total: 3.22s	remaining: 35m 41s
3:	learn: 3.7669983	test: 3.7575477	best: 3.7575477 (3)	total: 4.23s	remaining: 35m 12s
4:	learn: 3.7431149	test: 3.7301491	best: 3.7301491 (4)	total: 5.25s	remaining: 34m 53s
5:	learn: 3.7182841	test: 3.7040563	best: 3.7040563 (5)	total: 6.25s	remaining: 34m 38s
6:	learn: 3.6992247	test: 3.6858042	best: 3.6858042 (6)	total: 7.26s	remaining: 34m 27s
7:	learn: 3.6740373	test: 3.6657453	best: 3.6657453 (7)	total: 8.27s	remaining: 34m 20s
8:	learn: 3.6526294	test: 3.6463481	best: 3.6463481 (8)	total: 9.28s	remaining: 34m 12s
9:	learn: 3.6326525	test: 3.6284011	best: 3.6284011 (9)	total: 10.4s	remaining: 34m 27s
10:	learn: 3.6156900	test: 3.6143955	best: 3.6143955 (10)	total: 11.5s	remaining: 34m 32s
1

In [7]:
cat_clf.save_model('best_model/cat_clf')

### Torch NN

In [59]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [60]:
class ImageModel(nn.Module):
    def __init__(self):
        super(ImageModel, self).__init__()
        # self.flatten = nn.Flatten()
        self.linear_stack = nn.Sequential(
            nn.Linear(1344, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 50)
        )
    def forward(self, x):
        # x = self.flatten(x)
        logits = self.linear_stack(x)
        pred_proba = nn.Softmax(dim=1)(logits)
        return logits

image_model = ImageModel().to(device)

lr = 1e-4
batch_size = 512
epochs = 50

# loss function
loss_fn = nn.CrossEntropyLoss()

# optimizers
optimizer = torch.optim.Adam(image_model.parameters(), lr=lr)

In [61]:
# tensor
train_X = torch.from_numpy(train_X).to(torch.float32)
train_y = torch.from_numpy(train_y)

val_X = torch.from_numpy(val_X).to(torch.float32)
val_y = torch.from_numpy(val_y)

test_X = torch.from_numpy(test_X).to(torch.float32)
test_y = torch.from_numpy(test_y)

# dataset
train_dataset = TensorDataset(train_X, train_y)
val_dataset = TensorDataset(val_X, val_y)
test_dataset = TensorDataset(test_X, test_y)

# dataloader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [62]:
# training loop
def train_loop(epochs, model, loss_fn, optimizer, train_loader, val_loader=None):
    global device
    ls_loss = []
    ls_acc = []
    ls_val_loss = []
    dict_best = {'epoch': -1, 'best_val_loss': 1e10}
    
    model.train()
    for i in range(epochs):
        data_len = len(train_loader.dataset)
        loop = tqdm(enumerate(train_loader), total=len(train_loader))
        total_loss = 0
        correct_num = 0
        for batch_i, (X, y) in loop:
            
            X = X.to(device)
            y = y.to(device)
            # forward
            pred = model(X)
            loss = loss_fn(pred, y)

            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            correct_num += (pred.argmax(dim=1) == y).sum().item()
            total_loss += loss.item() * X.shape[0]
            # tqdm
            loop.set_description(f"Epochs: [{i+1}/{epochs}]")

            if (batch_i+1 == len(train_loader)):
                # last batch finish
                total_loss = total_loss / data_len
                acc_training = correct_num / data_len
                loop.set_postfix(loss=total_loss, acc=acc_training)
                ls_loss.append(total_loss)
                ls_acc.append(acc_training)

                # (optional) val loop
                if val_loader != None:
                    _, val_loss = val_stage(model, loss_fn, val_loader)
                    if val_loss < dict_best['best_val_loss']:
                        dict_best['best_val_loss'] = val_loss
                        dict_best['epoch'] = (i+1)
                        # save best model
                        torch.save(model, 'best_model/torch_nn.pth')
                    
                    loop.set_postfix(acc=acc_training, loss=total_loss, val_loss=val_loss)
                    ls_val_loss.append(val_loss)
    print(dict_best)
    return ls_acc, ls_loss, ls_val_loss
        

# val loop
def val_stage(model, loss_fn, val_loader):
    global device
    model.eval()
    with torch.no_grad():
        total_loss = 0
        correct_num = 0
        for batch_i, (X, y) in enumerate(val_loader):
            X, y = X.to(device), y.to(device)
            pred = model(X)

            correct_num += (pred.argmax(dim=1) == y).sum().item()
            total_loss += loss_fn(pred, y).item() * X.shape[0]
        total_acc = correct_num / len(val_loader.dataset)
        total_loss /= len(val_loader.dataset)
        return total_acc, total_loss

In [63]:
ls_acc, ls_loss, ls_val_loss = train_loop(epochs, image_model, loss_fn, optimizer, train_loader, val_loader)

Epochs: [50/50]: 100%|██████████| 124/124 [00:01<00:00, 104.77it/s, acc=0.674, loss=1.21, val_loss=4.79]

{'epoch': 10, 'best_val_loss': 3.2810845375061035}


In [64]:
# 轉回 numpy
train_y = train_y.numpy()
val_y = val_y.numpy()
test_y = test_y.numpy()

In [67]:
image_model = torch.load('best_model/torch_nn.pth')
val_y_proba = image_model(val_X.to(device)).cpu().detach().numpy()
test_y_proba = image_model(test_X.to(device)).cpu().detach().numpy()

val_eval = evaluate_score(val_y, val_y_proba)
test_eval = evaluate_score(test_y, test_y_proba)

print(val_eval)
print(test_eval)

# final model
# (0.16, 0.3956)
# (0.1578, 0.3822)

# best model
# (0.1733, 0.4133)
# (0.1778, 0.4222)


(0.1733, 0.4133)
(0.1778, 0.4222)
